In [ ]:
# Import the necessary libraries
import pandas as pd
import xgboost as xgb 
import pickle 
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer 
import gradio as gr

# Load your trained model 
with open('/Users/olivia/DS/tfidf_cutdown_model.pkl', 'rb') as model_file:
       model = pickle.load(model_file)


In [ ]:
# Removing Stopwords from the tweets
special_characters = ['~', ':', "'", '+', '[', '\\', '@', '^',
                      '{', '%', '(', '-', '"', '*', '|', ',', '&', '<', '`', '}', '.', '_', '=', ']', '>', ';', '#', '$', ')','!','?', '/', '’', '“', '”', "…"]

def replace_special(myemolist, myspeciallist):
    for i in myspeciallist:
        for j in range(len(myemolist)):
            myemolist[j] = myemolist[j].replace(i, "")
    return myemolist

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')


def identity_tokenizer(text):
    return text

tfidf = TfidfVectorizer(tokenizer=identity_tokenizer, lowercase=False, max_features = 1000)  

In [ ]:
import pickle

# Load the fitted vectorizer from the saved file
with open('/Users/olivia/DS/tfidf_vectorizer.pkl', 'rb') as vectorizer_file:
    tfidf = pickle.load(vectorizer_file)



In [ ]:
#tokenized, separated the emojis
from nltk.tokenize.casual import TweetTokenizer
t = TweetTokenizer()

In [ ]:
'''Create a predict function that takes a string as an input, processes the string in the same way that we did before
training our model. Returns the predicted age group of the user who wrote the string.
'''
def predict(text):
    try:
        # Preprocess the input text
        text = str(text)
        text = replace_special([text], special_characters)[0]
        text_df = pd.DataFrame({'text': [text]})
        text_df['text'] = text_df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
        text_df['tokenized'] = text_df.apply(lambda x: t.tokenize(x['text']), axis=1)
        tokenized_tweet = list(text_df['tokenized'])

        # Compute the TF-IDF representation of the preprocessed text
        tf_tweets = tfidf.transform(tokenized_tweet)
        tfidf_array = tf_tweets.toarray()
        prediction = model.predict(tfidf_array)
        if prediction[0] == 0:
            return('18-25')
        if prediction[0] == 1:
            return('26-40')
        if prediction[0] == 2:
            return('41-55')
        if prediction[0] == 3:
            return('56-70')
        if prediction[0] == 4:
            return('71+')

    except Exception as e:
        # Handle any exceptions raised by the function calls
        error_message = f"An error occurred during prediction: {str(e)}"
        return error_message

In [ ]:
# Define Gradio interface
output_text = gr.Textbox(label="Predicted age range: ")
title = 'Welcome to EmoJeneration! 👶🏻🧑🏻‍🦰👩🏻🧔🏼‍♂️👵🏽'
app_interface = gr.Interface(fn=predict, inputs= gr.Textbox(label = 'Enter some text:'), outputs=output_text, title = title)

# Launch the interface
app_interface.launch()